In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.simplefilter('ignore')

import gc

from os import path
import sys
sys.path.append(path.abspath('..'))

In [ ]:
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer

from timm import create_model
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

from src.pytorch import TorchDataset

In [ ]:
class LitTorch(LightningModule):
    def __init__(self):
        super().__init__()
        self.model = create_model(model_name='gernet_s', num_classes=5)
    
    def forward(self, x):
        return self.model(x)
    
    def process_batch(self, batch):
        return batch['image'], batch['label']
    
    def training_step(self, batch, batch_idx):
        x, y = self.process_batch(batch)
        logits = self(x)
        loss = F.binary_cross_entropy_with_logits(logits, y)
        return loss
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=1e-3)
    
    def prepare_data(self):
        self.dataset = TorchDataset()
        
    def train_dataloader(self):
        return DataLoader(self.dataset, batch_size=60, num_workers=4)

In [ ]:
model = LitTorch()

In [ ]:
trainer = Trainer(gpus=1, max_epochs=1)

In [ ]:
%%time
trainer.fit(model)

In [ ]:
# 3min 18s